https://medium.com/swlh/how-to-scrape-large-amounts-of-reddit-data-using-pushshift-1d33bde9286

In [10]:
import os
import sys
# //*** Imports and Load Data
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pmaw import PushshiftAPI

#//*** Use the whole window in the IPYNB editor
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

#//*** Maximize columns and rows displayed by pandas
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', None)


api = PushshiftAPI()

import datetime as dt
before = int(dt.datetime(2021,6,26,0,0).timestamp())
after = int(dt.datetime(2020,12,1,0,0).timestamp())

In [119]:
#//***************************************
#//*** Apply Common Cleanup operations
#//***************************************
#//*** In anticpation that I'll be re-using text cleanup code. I'm adding some robustness to the function.
#//*** Adding kwargs to disable features that default to true.
#//*** Whether an action is skipped or executed is based on a boolean value stored in action_dict.
#//*** Key values will default to true. If code needs to be defaulted to False, a default_false list can be added later
#//*** All Boolean kwarg keya are stored in kwarg list. This speeds up the coding of the action_dict.
#//*** As Kwargs are added 
def mr_clean_text(input_series, input_options={}):
    
    #//*** import time library
    try:
        type(time)
    except:
        import time
    
    #//*** Start Timing the process
    start_time = time.time()

    
    #//*** Add some data validation. I'm preparing this function for additional use. I'm checking if future users (ie future me)
    #//*** may throw some garbage at this function. Experience has taught me to fail safely wherever possible.

    #//*** All kwargs are listed here. These initialize TRUE by default.
    key_list = [ "lower", "newline", "html", "remove_empty", "punctuation" ]
    
    #//*** Build Action Dictionary
    action_dict = { } 
    
    #//*** Build the keys from kwarg_list and default them to TRUE
    for key in key_list:
        action_dict[key] = True
        
    #//*** Loop through the input kwargs (if any). Assign the action_dict values based on the kwargs:
    for key,value in input_options.items():
        print(key,value)
        action_dict[key] = value
    
    
    #//*************************************************************************
    #//*** The Cleanup/Processing code is a straight lift from DSC550 - Week02
    #//*************************************************************************
    #//*** Convert to Lower Case, Default to True
    if action_dict["lower"]:
        input_series = input_series.str.lower()
    
   
    #//*** Remove New Lines
    if action_dict["newline"]:
        #//*** Rmove \r\n
        input_series = input_series.str.replace(r'\r?\n',"")

        #//*** Remove \n new lines
        input_series = input_series.str.replace(r'\n',"")

    #//*** Remove html entities, observed entities are &gt; and &lt;. All HTML entities begin with & and end with ;.
    #//*** Let's use regex to remove html entities
    if action_dict["html"]:
        input_series = input_series.str.replace(r'&.*;',"")

    #//*** Remove the empty lines
    if action_dict["remove_empty"]:
        input_series = input_series[ input_series.str.len() > 0]

    #//*** Remove punctuation
    if action_dict["punctuation"]:
        #//*** Load libraries for punctuation if not already loaded.
        #//*** Wrapping these in a try, no sense in importing libraries that already exist.
        #//*** Unsure of the cost of reimporting libraries (if any). But testing if library is already loaded feels
        #//*** like a good practice
        try:
            type(sys)
        except:
            import sys

        try:
            type(unicodedata)
        except:
            import unicodedata
        
        #//*** replace Comma and Period with a space.
        for punct in [",",".","$"]:
            input_series = input_series.str.replace(punct," ")

        #//*** Remove punctuation using the example from the book
        punctuation = dict.fromkeys(i for i in range(sys.maxunicode) if unicodedata.category(chr(i)).startswith('P') )
        input_series = input_series.str.translate(punctuation)

    print(f"Text Cleaning Time: {time.time() - start_time}")

    return input_series
#//*** Remove Stop words from the input list
def remove_stop_words(input_series):
    
    #//*** This function removes stop_words from a series.
    #//*** Works with series.apply()
    def apply_stop_words(input_list):

        #//*** Load Stopwords   
        for word in input_list:
            if word in stop_words:
                input_list.remove(word)
        return input_list

    #//*** import nltk if needed
    try:
        type(nltk)
    except:
        import nltk
        
    stopwords = nltk.corpus.stopwords

    #//*** Stopwords requires an additional download
    try:
        type(stopwords)
    except:
        nltk.download('stopwords')


    #//*** import time library
    try:
        type(time)
    except:
        import time

    #//*** Start Timing the process
    start_time = time.time()


    #//*** The stop_words include punctuation. Stop Word Contractions will not be filtered out.
    stop_words = []

    #//*** Remove apostrophies from the stop_words
    for stop in stopwords.words('english'):
        stop_words.append(stop.replace("'",""))

    
    #//*** Remove Stop words from the tokenized strings in the 'process' column
    #input_series = input_series.apply(remove_stop_words,stop_words)
    
    input_series = input_series.apply(apply_stop_words)

    print(f"Stop Words Time: {time.time() - start_time}")
    
    return input_series
#//*** Tokenize a Series containing Strings.
#//*** Breaking this out into it's own function for later reuse.
#//*** Not a lot of code here, but it helps to keep the libraries localized. This creates standarization for future
#//*** Stoneburner projects. Also has the ability to add functionality as needed.

def tokenize_series(input_series):
    
    try:
        type(nltk)
    except:
        import nltk
    
    word_tokenize = nltk.tokenize.word_tokenize 
    
    #//*** import time library
    try:
        type(time)
    except:
        import time
    
    #//*** Start Timing the process
    start_time = time.time()
    
    input_series = input_series.apply(word_tokenize)
    
    print(f"Tokenize Time: {time.time() - start_time}")
    
    return input_series


In [22]:
#//*** Download the First 100,000 Comments from reddit pushsift
subreddit="wallstreetbets"
limit=100000
#comments = api.search_comments(subreddit=subreddit, limit=limit, before=before after=after)
comments = api.search_comments(subreddit=subreddit, limit=limit, after=after)
print(f'Retrieved {len(comments)} comments from Pushshift')

#//*** Convert comments to Dataframe
comments_df = pd.DataFrame(comments)

#//*** Save DataFrame to a file for processing
comments_df.to_csv("pushshift_raw_comments.csv.zip",compression="zip",index=False)

Total:: Success Rate: 100.00% - Requests: 1011 - Batches: 102 - Items Remaining: 0
Retrieved 100000 comments from Pushshift


In [150]:
#//*** Build list of ticker symbols from NYSE and NASDAQ
#//*** Reads from Excel file.
#//*** Gets the Symbol column, and converts to lower case, 
nyse = pd.read_csv("NYSE_20210625.csv",header=None)[0].str.lower()
nasdaq = pd.read_csv("NASDAQ_20210625.csv",header=None)[0].str.lower()

#//*** Removes symbols with 1 and 2 character listings
nyse = list(nyse[nyse.apply(lambda x: len(x)>2) ])
nasdaq = list(nasdaq[nasdaq.apply(lambda x: len(x)>2) ])

symbols = nyse + nasdaq

In [151]:
print(symbols)

['aac', 'aac.u', 'aac.w', 'aai-b', 'aai-c', 'aaic', 'aan', 'aap', 'aaq.u', 'aaq.w', 'aaqc', 'aat', 'abb', 'abbv', 'abc', 'abev', 'abg', 'abm', 'abr', 'abr-d', 'abt', 'aca', 'acc', 'acco', 'acel', 'ach', 'aci', 'aci.s', 'aci.t', 'aci.u', 'aci.w', 'acic', 'acii', 'acm', 'acn', 'acn.u', 'acn.w', 'acnd', 'acp', 'acp-a', 'acr', 'acr-c', 'acr-d', 'acre', 'acv', 'adc', 'adct', 'ade.u', 'ade.w', 'adf', 'adf.w', 'adm', 'adnt', 'ads', 'adt', 'adx', 'aeb', 'aee', 'aefc', 'aeg', 'ael', 'ael-a', 'ael-b', 'aem', 'aenz', 'aeo', 'aer', 'aes', 'aesc', 'aev.w', 'aeva', 'afb', 'afg', 'afgb', 'afgc', 'afgd', 'afge', 'afi', 'afl', 'aft', 'aga.u', 'aga.w', 'agac', 'agcb', 'agco', 'agd', 'agi', 'agl', 'agm', 'agm-c', 'agm-d', 'agm-e', 'agm-f', 'agm-g', 'ago', 'ago-b', 'ago-e', 'ago-f', 'agr', 'agro', 'ags', 'agti', 'agx', 'ahc', 'ahh', 'ahh-a', 'ahl-c', 'ahl-d', 'ahl-e', 'aht', 'aht-d', 'aht-f', 'aht-g', 'aht-h', 'aht-i', 'aic', 'aif', 'aig', 'aig-a', 'ain', 'aio', 'air', 'airc', 'ait', 'aiv', 'aiz', 'aizn',

In [162]:

raw_df = pd.read_csv("pushshift_raw_comments.csv.zip",compression="zip")

In [187]:
#//*** Read Dataframe from file
raw_df = pd.read_csv("pushshift_raw_comments.csv.zip",compression="zip")

#//*** Keep the important columns
raw_df = raw_df[["score","total_awards_received","created_utc","is_submitter","author_fullname","body","id","link_id","parent_id","distinguished","stickied","permalink","retrieved_on","subreddit","subreddit_id"]]

#//*** Clean text, tokenize and remove stop words
raw_df['clean'] = remove_stop_words(tokenize_series(mr_clean_text(raw_df['body'],{"remove_empty":False})))
raw_df['hash'] = raw_df['body'].apply(hash)
raw_df

remove_empty False
Text Cleaning Time: 1.2832801342010498
Tokenize Time: 11.41757345199585
Stop Words Time: 1.5213427543640137


score  total_awards_received  created_utc  is_submitter  \
0          2                      0   1608602327         False   
1          1                      0   1608602326         False   
2          3                      0   1608602326         False   
3          7                      0   1608602324         False   
4          1                      0   1608602322         False   
...      ...                    ...          ...           ...   
99995      7                      0   1618712791         False   
99996      2                      0   1618712790         False   
99997      1                      0   1618712790         False   
99998      3                      0   1618712787         False   
99999      1                      0   1618712779         False   

      author_fullname                                               body  \
0         t2_44w4c3vn  No longer know if what I own is good. The move...   
1            t2_ohs9s         Why is Apple worth 60% more now than June?   
2            t2_e6q9p  So are DOW and SPY just gonna chill red until ...   
3         t2_36t07oui  I’ve heard Apple Car rumors for years now. Are...   
4         t2_49h1we29                                       LGVW!!!! 🦋🦋🦋   
...               ...                                                ...   
99995     t2_7xc4t7qe              Watching planet of the apes right now   
99996       t2_123o5k  My $RIDE April 30 C10 aren’t gonna print are t...   
99997        t2_6rut2  I don't know who either of these idiots are in...   
99998     t2_a57oe8f9  Damn, I had 4/23 calls.  Guess I'm selling tho...   
99999             NaN                                          [removed]   

            id    link_id   parent_id distinguished  stickied  \
0      ggndpyd  t3_khq3x2   t3_khq3x2           NaN     False   
1      ggndpwg  t3_khq3x2  t1_ggnde5q           NaN     False   
2      ggndpvb  t3_khq3x2   t3_khq3x2           NaN     False   
3      ggndprr  t3_khq3x2   t3_khq3x2           NaN     False   
4      ggndply  t3_khq3x2  t1_ggndhdz           NaN     False   
...        ...        ...         ...           ...       ...   
99995  guxfdxm  t3_msb6mb   t3_msb6mb           NaN     False   
99996  guxfduk  t3_msb6mb   t3_msb6mb           NaN     False   
99997  guxfdua  t3_msb6mb   t3_msb6mb           NaN     False   
99998  guxfdow  t3_mt42h8   t3_mt42h8           NaN     False   
99999  guxfd7y  t3_msv5x6   t3_msv5x6           NaN     False   

                                               permalink  retrieved_on  \
0      /r/wallstreetbets/comments/khq3x2/what_are_you...    1608608690   
1      /r/wallstreetbets/comments/khq3x2/what_are_you...    1608608689   
2      /r/wallstreetbets/comments/khq3x2/what_are_you...    1608608689   
3      /r/wallstreetbets/comments/khq3x2/what_are_you...    1608608687   
4      /r/wallstreetbets/comments/khq3x2/what_are_you...    1608608685   
...                                                  ...           ...   
99995  /r/wallstreetbets/comments/msb6mb/weekend_disc...    1618719711   
99996  /r/wallstreetbets/comments/msb6mb/weekend_disc...    1618719710   
99997  /r/wallstreetbets/comments/msb6mb/weekend_disc...    1618719710   
99998  /r/wallstreetbets/comments/mt42h8/what_does_to...    1618719707   
99999  /r/wallstreetbets/comments/msv5x6/himx_yolo_my...    1618719700   

            subreddit subreddit_id  \
0      wallstreetbets     t5_2th52   
1      wallstreetbets     t5_2th52   
2      wallstreetbets     t5_2th52   
3      wallstreetbets     t5_2th52   
4      wallstreetbets     t5_2th52   
...               ...          ...   
99995  wallstreetbets     t5_2th52   
99996  wallstreetbets     t5_2th52   
99997  wallstreetbets     t5_2th52   
99998  wallstreetbets     t5_2th52   
99999  wallstreetbets     t5_2th52   

                                                   clean                 hash  
0      [longer, know, what, own, good, moves, so, wil...   320585662671538714  
1             

In [191]:
#//*** Count each Stock mention add it to a dictionary of lists. Each list is filled with 0s. The Specific row index is updated with the relevant count. 
#//*** This Generates a word count matrix
stock_dict = {}

#//*** Keep Track of Rows
index = 0

for row in raw_df.iterrows():
    
    #//*** Get the cleaned body text
    body = row[1]['clean']
    
    #//*** For Each Stock Symbol
    for stock in symbols:
        
        #//*** Check if Stock exists in Body
        if stock in body:
            
            #//*** Reset the stock counter
            count = 0
            
            #//*** Loop through body and county ticker mentions
            for word in body:
                #//*** If word found increment count
                if stock == word:
                    count += 1
                    
            #//*** Check if symbol is in stock_dict
            if stock not in stock_dict.keys():    

                #//*** If not, then build it
                stock_dict[stock] = np.zeros(len(raw_df))
            
            #//*** Update the stock value at the 
            stock_dict[stock][index] = count

    #//*** Increment Index to keep with row index
    index +=1   

#//*** Loop through the dictionary key and lists
for col,values in stock_dict.items():
    
    #//*** Add each key (which is a stock ticker symbol) as a column using the list of ticker counts for Data
    raw_df[col] = values.astype('int')
    
raw_df

score  total_awards_received  created_utc  is_submitter  \
0          2                      0   1608602327         False   
1          1                      0   1608602326         False   
2          3                      0   1608602326         False   
3          7                      0   1608602324         False   
4          1                      0   1608602322         False   
...      ...                    ...          ...           ...   
99995      7                      0   1618712791         False   
99996      2                      0   1618712790         False   
99997      1                      0   1618712790         False   
99998      3                      0   1618712787         False   
99999      1                      0   1618712779         False   

      author_fullname                                               body  \
0         t2_44w4c3vn  No longer know if what I own is good. The move...   
1            t2_ohs9s         Why is Apple worth 60% more now than June?   
2            t2_e6q9p  So are DOW and SPY just gonna chill red until ...   
3         t2_36t07oui  I’ve heard Apple Car rumors for years now. Are...   
4         t2_49h1we29                                       LGVW!!!! 🦋🦋🦋   
...               ...                                                ...   
99995     t2_7xc4t7qe              Watching planet of the apes right now   
99996       t2_123o5k  My $RIDE April 30 C10 aren’t gonna print are t...   
99997        t2_6rut2  I don't know who either of these idiots are in...   
99998     t2_a57oe8f9  Damn, I had 4/23 calls.  Guess I'm selling tho...   
99999             NaN                                          [removed]   

            id    link_id   parent_id distinguished  stickied  \
0      ggndpyd  t3_khq3x2   t3_khq3x2           NaN     False   
1      ggndpwg  t3_khq3x2  t1_ggnde5q           NaN     False   
2      ggndpvb  t3_khq3x2   t3_khq3x2           NaN     False   
3      ggndprr  t3_khq3x2   t3_khq3x2           NaN     False   
4      ggndply  t3_khq3x2  t1_ggndhdz           NaN     False   
...        ...        ...         ...           ...       ...   
99995  guxfdxm  t3_msb6mb   t3_msb6mb           NaN     False   
99996  guxfduk  t3_msb6mb   t3_msb6mb           NaN     False   
99997  guxfdua  t3_msb6mb   t3_msb6mb           NaN     False   
99998  guxfdow  t3_mt42h8   t3_mt42h8           NaN     False   
99999  guxfd7y  t3_msv5x6   t3_msv5x6           NaN     False   

                                               permalink  retrieved_on  \
0      /r/wallstreetbets/comments/khq3x2/what_are_you...    1608608690   
1      /r/wallstreetbets/comments/khq3x2/what_are_you...    1608608689   
2      /r/wallstreetbets/comments/khq3x2/what_are_you...    1608608689   
3      /r/wallstreetbets/comments/khq3x2/what_are_you...    1608608687   
4      /r/wallstreetbets/comments/khq3x2/what_are_you...    1608608685   
...                                                  ...           ...   
99995  /r/wallstreetbets/comments/msb6mb/weekend_disc...    1618719711   
99996  /r/wallstreetbets/comments/msb6mb/weekend_disc...    1618719710   
99997  /r/wallstreetbets/comments/msb6mb/weekend_disc...    1618719710   
99998  /r/wallstreetbets/comments/mt42h8/what_does_to...    1618719707   
99999  /r/wallstreetbets/comments/msv5x6/himx_yolo_my...    1618719700   

            subreddit subreddit_id  \
0      wallstreetbets     t5_2th52   
1      wallstreetbets     t5_2th52   
2      wallstreetbets     t5_2th52   
3      wallstreetbets     t5_2th52   
4      wallstreetbets     t5_2th52   
...               ...          ...   
99995  wallstreetbets     t5_2th52   
99996  wallstreetbets     t5_2th52   
99997  wallstreetbets     t5_2th52   
99998  wallstreetbets     t5_2th52   
99999  wallstreetbets     t5_2th52   

                                                   clean                 hash  \
0      [longer, know, what, own, good, moves, so, wil...   320585662671538714   
1           

In [192]:
raw_df.to_csv("reddit_comments.csv.zip",compression="zip",index=False)